In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../data"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1)

# Any results you write to the current directory are saved as output.

2024-05-30 12:43:20.249177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 12:43:20.249324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 12:43:20.490224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 12:43:21.188728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 12:43:22.150406: W tensorflow/compiler/tf2

['spam_classification.csv', 'Hindi_English_Truncated_Corpus.csv', 'pneumonia']


In [2]:
lines=pd.read_csv("../data/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
lines['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [4]:
#lines=lines[lines['source']=='ted']

In [5]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [6]:
pd.isnull(lines).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [7]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [8]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [9]:
lines=lines.sample(n=20000,random_state=42)
lines.shape

(20000, 3)

In [10]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [11]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [14]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.head()

,source,english_sentence,hindi_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END


In [16]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
len(all_eng_words)

27301

In [18]:
all_eng_words

{'acknowledge',
 'arthashastra',
 'karakoram',
 'yogis',
 'secondhand',
 'modelling',
 'addon',
 'athatva',
 'poetseer',
 'permission',
 'corrugated',
 'mummy',
 'tibetians',
 'income',
 'abductors',
 'shehnai',
 'comparitively',
 'bihari',
 'flourished',
 'bangles',
 'hydroelectricity',
 'dwarikapith',
 'vayu',
 'inconspicuous',
 'inlets',
 'filed',
 'calle',
 'internetinternet',
 'kaikeyi',
 'length',
 'reelected',
 'reflex',
 'apartheid',
 'eaxmple',
 'electric',
 'incompatibility',
 'proses',
 'excavations',
 'aswathama',
 'dvaragopura',
 'winston',
 'ali',
 'cactus',
 'tongues',
 'marim',
 'birley',
 'retracting',
 'auli',
 'misappropriation',
 'mathias',
 'monsoons',
 'organisations',
 'gangaikondacholapuram',
 'circus',
 'postponement',
 'yadava',
 'stationsits',
 'parnell',
 'shahjehan',
 'nepali',
 'rewriting',
 'alak',
 'earthquakes',
 'obstacles',
 'shops',
 'coalition',
 'upwards',
 'synthesised',
 'solar',
 'pillankuzhal',
 'abul',
 'cer',
 'kill',
 'metres',
 'industriali

In [19]:
len(all_hindi_words)

32710

In [20]:
all_hindi_words

{'शौचादि',
 'शाब्दिक',
 'जनतात्रिक',
 'शुल्क',
 'नाम',
 'कर्तवय',
 'लैप्प्ली',
 'महत्व',
 'भित्तिलेखप्रेमचंद',
 'रजत',
 'ऋमांक',
 'थ्रो',
 'शिखर',
 'हैदेखने',
 'वैतरणी',
 'घपला',
 'गन्धर्व',
 'कटोरा',
 'छोटेसे',
 'चर्चगेट',
 'मनसेमन',
 'आलोकित',
 'हिन्दुत्व',
 'ब्रोंक्स',
 'वफादारी',
 'सुखशांति',
 'ड़ालेंगे',
 'कर्नल',
 'जटाओं',
 'सोनार',
 'व्याकुलता',
 'परिभाषित',
 'नोइ',
 'रविदास',
 'वेल्लोर',
 'फैक्ट्री',
 'electric',
 'गंभीरता',
 'नॉक़्टुइड',
 'अनिर्णीत',
 'सिर्फ़',
 'चर्चा',
 'सिद्धार्थ',
 'हरिव्यासी',
 'फूस',
 'हॉल',
 'याकूट',
 'पददलितों',
 'सलीमगढ',
 'प्रतियोगिता',
 'कॉमनवैल्थ',
 'भावातिरेक',
 'एअरवेज',
 'अब्को',
 'kham',
 'समस्याए',
 'तीसमारखाँ',
 'प्रभुसत्ता',
 'ट्रेफिक',
 'नुमा',
 'श्री',
 'गुरुदेव',
 'जन्मतिथि',
 'स्टैंडिंग',
 'पट्टीं',
 'परिचर',
 'अन्दर',
 'हमले',
 'षडयन्त्र',
 'तब्दीली',
 'चील।',
 'बारें',
 'स्पार्टन',
 'श्रेष्ठियों',
 'ऑटो',
 'श्रृंखला',
 'तात्विक',
 'ज़्योतिष',
 'लिपिक',
 'वहनीय',
 'निर्बंध',
 'चौंध',
 'डिब्बा',
 'सवेरा',
 'शर्तों',
 'सम्माननीय',
 'दिव्य

In [21]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [22]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END,14,21
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END,22,24


In [23]:
lines[lines['length_eng_sentence']>30].shape

(1984, 5)

In [24]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [25]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
57202,ted,there was lasagna there was casseroles,START_ वहां लाजान्या था कैसेरोल थे _END,6,7
107821,indic2012,super power india source google writer vedpratap vedik,START_ महाशक्ति भारत गूगल पुस्तक लेखक वेदप्रताप वैदिक _END,8,9
...,...,...,...,...,...
77124,tides,the walls should be whitewashed with lime containing ddt,START_ दीवार पर सफेदी कर दी जानी चाहिएडी डी टी युकऋऊण्श्छ्ष्त चूने से सफेदी की जानी चाहिए _END,9,18
16275,indic2012,sim card,START_ सिम कार्ड _END,2,4
32908,indic2012,footpath behind the mirror image of lake,START_ प्रतिबिम्बित झील के पीछे पैदल पथ _END,7,8
36605,ted,it was called the vaitarani,START_ इस नदी का नाम है वैतरणी । _END,5,9


In [26]:
lines.shape

(13193, 5)

In [27]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [28]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [29]:
print(max_length_src)

20


In [30]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(27301, 32710)

In [31]:
num_decoder_tokens += 1 #for zero padding


In [32]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [33]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [34]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
44173,tides,sheep produce two different kinds of crops each year wool and lambs,START_ भेड़ों से प्रतिवर्ष ऊन तथा मेमने प्राप्त होते हैं _END,12,11
125550,ted,superman wonder woman and their colleagues,START_ सुपरमैन वंडर वुमन और उनके बाकी साथी _END,6,9
43451,indic2012,lakhimpur kheri district,START_ लखिमपुर खेरी जिला _END,3,5
28242,tides,most of them belong to the poor strata of society,START_ इनमें अधिक लोग गरीब वर्ग के हैं _END,10,9
112176,tides,it is true that many authors have composed beautiful vachanas,START_ यह सच है कि कईयों ने सुन्दर वचनों की रचना की है _END,10,14
72387,ted,tackling the issues affecting their country,START_ जिसमें उन्हें उनके देश की समस्याओं का समाधान _END,6,10
91089,indic2012,in his life he got “”speaker king“” reward,START_ उन्हें अपने जीवन काल में ही ‘उपन्यास सम्राट की पदवी मिल गयी थी। _END,8,15
57581,indic2012,valmiki ramayana computer test,START_ वाल्मीकिरामायण कम्प्यूटर की कसौटी पर _END,4,7
118770,tides,in response to these requests an autonomous body called the indian parliamentary group was set up,START_ ऐसे अनुरोध किये जाने पर “ भारतीय संसदीय ग्रुप ” नामक एक स्वायत्त निकाय स्थापित किया गया _END,16,19
126430,ted,the second one they were actually split,START_ और दूसरे वाले में अलग अलग विचार थे _END,7,10


### Split the data into train and test

In [35]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((10554,), (2639,))

In [36]:
X_train

36915     and their leaders their leaders                                                
123666    i looked at her and knew that she was ruthless in her beauty                   
42144     at that time velayuda mudaliar was a tamil pundit in presidency college madras 
95115     i know i didnt and i still dont                                                
27386     hopefully youll see the film                                                   
                      ...                                                                
41275     these assistants were called naib qazis                                        
109995    unfortunatelyhe failed to complete it properly                                 
14512     all right now that might seem like an even taller order to actually demonstrate
25367     bread                                                                          
86195     youll hear a train in moment that they dont react to                           
Name: engl

In [37]:
y_train

36915     START_ और उनके नेताउनके लीडर _END                                                 
123666    START_ मैंने उसकी तरफ देखा और पाया कि वह अपने सौंदर्य में निष्ठुर थी _END         
42144     START_ उस समय वेलायुद मुदलियार मद्रास के प्रेसिडेंसी कालज मे तमिल के पंडित थे _END
95115     START_ मैं जानती हूँ कि मैं नहीं करती थी और अब भी नहीं करती हूँ _END              
27386     START_ उम्मीद है आप फिल्म देखेंगे _END                                            
                           ...                                                              
41275     START_ इन सहायकों को नायब काजी कहते थे _END                                       
109995    START_ उनके दुर्भाग्य से वे इसे ढंग से पूरा नहीं कर पाये। _END                    
14512     START_ ये सत्य है कि इस बात को असल में दिखा पाना बहुत कठिन काम होगा। _END         
25367     START_ अप्पम _END                                                                 
86195     START_ आप कुछ देर में ट्रेन की आवाज़ सुनेंगे जिस पर ये प्रति

### Let us save this data

In [38]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [39]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [40]:
latent_dim=300

In [41]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

2024-05-30 12:43:26.500206: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-30 12:43:27.062833: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-30 12:43:27.063081: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [42]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [44]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            8190300   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            9813300   ['input_2[0][0]']             
                                                                                              

In [45]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 10

In [46]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



/tmp/ipykernel_572326/2713035456.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/10


2024-05-30 12:43:29.935249: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 334960640 exceeds 10% of free system memory.
2024-05-30 12:43:33.980017: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 334960640 exceeds 10% of free system memory.
2024-05-30 12:43:34.245615: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 334981120 bytes after encountering the first element of size 334981120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-05-30 12:43:34.363448: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 334960640 exceeds 10% of free system memory.
2024-05-30 12:43:34.469880: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected com

 2/82 [..............................] - ETA: 43s - loss: 10.3944 

2024-05-30 12:43:37.008996: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 334960640 exceeds 10% of free system memory.


...

82/82 [==============================] - 44s 540ms/step - loss: 6.7877 - val_loss: 6.9289
Epoch 8/10
82/82 [==============================] - ETA: 0s - loss: 6.7723

2024-05-30 12:49:24.692920: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 334981120 bytes after encountering the first element of size 334981120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


82/82 [==============================] - 45s 556ms/step - loss: 6.7723 - val_loss: 6.9336
Epoch 9/10
82/82 [==============================] - 46s 556ms/step - loss: 6.7595 - val_loss: 6.9238
Epoch 10/10
82/82 [==============================] - ETA: 0s - loss: 6.7454

2024-05-30 12:50:55.643452: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 334981120 bytes after encountering the first element of size 334981120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


82/82 [==============================] - 46s 561ms/step - loss: 6.7454 - val_loss: 6.9294


In [47]:
#model.save_weights('nmt_weights.h5')


In [48]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [49]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [50]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [51]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 14ms/step
Input English sentence: and their leaders their leaders
Actual Hindi Translation:  और उनके नेताउनके लीडर 
Predicted Hindi Translation:  और के के के 


In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 15ms/step
Input English sentence: i looked at her and knew that she was ruthless in her beauty
Actual Hindi Translation:  मैंने उसकी तरफ देखा और पाया कि वह अपने सौंदर्य में निष्ठुर थी 
Predicted Hindi Translation:  और के के के के के के के के के 


In [53]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 14ms/step
Input English sentence: at that time velayuda mudaliar was a tamil pundit in presidency college madras
Actual Hindi Translation:  उस समय वेलायुद मुदलियार मद्रास के प्रेसिडेंसी कालज मे तमिल के पंडित थे 
Predicted Hindi Translation:  और के के के के के के के के के 


In [54]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 14ms/step
Input English sentence: i know i didnt and i still dont
Actual Hindi Translation:  मैं जानती हूँ कि मैं नहीं करती थी और अब भी नहीं करती हूँ 
Predicted Hindi Translation:  और के के के के के 


In [55]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 14ms/step
Input English sentence: hopefully youll see the film
Actual Hindi Translation:  उम्मीद है आप फिल्म देखेंगे 
Predicted Hindi Translation:  और के के के 


In [56]:
#model.save('models/english_to_hindi_translator.h5')

In [57]:
# Save the encoder and decoder models
encoder_model.save('../models/english_to_hindi_translator/encoder_model_e2h.h5')
decoder_model.save('../models/english_to_hindi_translator/decoder_model_e2h.h5')


/home/roronoa/ai/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
import pickle

# Save the tokenizers
with open('../models/english_to_hindi_translator/english_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(input_token_index, f)

with open('../models/english_to_hindi_translator/hindi_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(target_token_index, f)

# Save the reverse tokenizers
with open('../models/english_to_hindi_translator/reverse_english_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(reverse_input_char_index, f)

with open('../models/english_to_hindi_translator/reverse_hindi_tokenizer_e2h.pkl', 'wb') as f:
    pickle.dump(reverse_target_char_index, f)


In [62]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
import pickle

# Load the models
encoder_model = load_model('../models/english_to_hindi_translator/encoder_model_e2h.h5')
decoder_model = load_model('../models/english_to_hindi_translator/decoder_model_e2h.h5')

# Load the tokenizers
with open('../models/english_to_hindi_translator/english_tokenizer_e2h.pkl', 'rb') as f:
    input_token_index = pickle.load(f)

with open('../models/english_to_hindi_translator/hindi_tokenizer_e2h.pkl', 'rb') as f:
    target_token_index = pickle.load(f)

with open('../models/english_to_hindi_translator/reverse_english_tokenizer_e2h.pkl', 'rb') as f:
    reverse_input_char_index = pickle.load(f)

with open('../models/english_to_hindi_translator/reverse_hindi_tokenizer_e2h.pkl', 'rb') as f:
    reverse_target_char_index = pickle.load(f)

max_length_src = 20  # Set this to the maximum length used during training
latent_dim = 300
num_decoder_tokens = len(target_token_index) + 1

def translate_sentence(input_sentence):
    input_seq = [input_token_index.get(word, 0) for word in input_sentence.split()]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=max_length_src, padding='post')

    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        # Exit condition: either hit max length or find stop character
        if sampled_char == '_END' or len(decoded_sentence) > max_length_src:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_char)

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return ' '.join(decoded_sentence)

# Example usage
print(translate_sentence("hopefully youll see the film"))


1/1 [==============================] - 0s 14ms/step
और के के के
